In [1]:
import pandas as pd
import json
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from PIL import Image
from nltk.tokenize import word_tokenize
from gensim.models import FastText
from gensim.models.fasttext import load_facebook_model
from ydata_profiling import ProfileReport
import plotly.express as px
from tabulate import tabulate
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
import re
from nltk.stem import PorterStemmer
from collections import Counter
import random
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
nltk.download('punkt') 
nltk.download('stopwords')   
nltk.download('wordnet')  
nltk.download('punkt', download_dir='/kaggle/working/nltk_data/')
nltk.download('stopwords', download_dir='/kaggle/working/nltk_data/')
nltk.download('wordnet', download_dir='/kaggle/working/nltk_data/')
nltk.data.path.append('/kaggle/working/nltk_data/')
import warnings
warnings.filterwarnings('ignore')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

stemmer = PorterStemmer()


2024-05-05 01:11:44.750778: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-05 01:11:44.750958: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-05 01:11:44.907589: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /kaggle/working/nltk_data/...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /kaggle/working/nltk_data/...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data/...


In [2]:
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
dataset_path="/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv"
df = pd.read_csv(dataset_path, encoding =DATASET_ENCODING , names=DATASET_COLUMNS,index_col=False)
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
df=df[["target","text"]]
df.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [4]:
# Plot distribution of sentiment classes
fig = px.bar(df['target'].value_counts(), title='Sentiment Distribution',
             labels={'value': 'Number of Tweets', 'index': 'Sentiment'}, 
             text_auto=True,
             template='plotly_dark')

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)'
)

fig.show()

In [5]:
first_half = df.iloc[:80000]
second_half = df.iloc[80000:]

# Randomly sample 100 rows from each half
sample_first_half = first_half.sample(200)  # random_state for reproducibility
sample_second_half = second_half.sample(200)

# Concatenate the two samples into a new DataFrame
df = pd.concat([sample_first_half, sample_second_half])
df.head(10)

,target,text
77964,0,I am home from the job I was so close to quitt...
46211,0,@Oofxx Well it's still kind of friday night fo...
6584,0,Never taking prescribed pain meds again. Still...
61969,0,the people in city I live hate me!! wanne go a...
69525,0,I need a new journal
47244,0,"I was meant to be out of bed at 9, but no I we..."
19937,0,@Wakeangl I know. I was pretty psyched myself...
65726,0,@abowllan lol ..ummm YOU??? hahaha so sweet bu...
33532,0,"At work bleh... 9pm now, 10 hours to go"
75765,0,tired have been on facebook for the whole nigh...


In [6]:
def preprocess_text(text):
        text = text.lower()
        text = re.sub(r'@\w+', '', text)
        text = re.sub(r'[^a-z\s]', '', text)
        text=re.sub(r'\w*\d\w*','',text)
        
        tokens = word_tokenize(text)
        stop_words = set(stopwords.words('english'))
        stemmer = PorterStemmer()
        
        filtered_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
        
        return " ".join(filtered_tokens)
    
    

In [7]:
def get_max_lenght(filtered_tokens):
    return max(len(word) for sentence in filtered_tokens for word in sentence.split())
    

In [8]:
def padding_text(filtered_tokens,maxlen):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([filtered_tokens]) 
    sequences = tokenizer.texts_to_sequences([filtered_tokens])
    
    padded_sequences = pad_sequences(sequences, maxlen=maxlen, padding='post')
    return padded_sequences
    
    

In [9]:
df['text']=df['text'].apply(preprocess_text)
max_len=get_max_lenght(df['text'])
df['padding']=df['text'].apply(lambda tokens:padding_text(tokens,max_len))

In [10]:
def print_top_last_10(text):
    allwords=[word for sentance in text for word in sentance.split()]
    print("=== Top 10 Words =====" )
    print(tabulate(Counter(allwords).most_common()[:10],headers=['word','freq'],tablefmt="fancy_grid"))
    print("=== Last 10 Words =====" )
    print(tabulate(Counter(allwords).most_common()[::-1][:10],headers=['word','freq'],tablefmt="fancy_grid"))
    